In [1]:
import spacy
from spacy.training import Example
from spacy.util import minibatch, compounding
import random
import json
from tqdm import tqdm
from spacy.util import filter_spans

In [2]:
with open('Data/train/train_data.json','rb') as f:
    train_data=json.load(f)

In [ ]:
# create a blank English NLP model
nlp = spacy.blank('en')

# Create the NER component and add it to the pipeline
if "ner" not in nlp.pipe_names:
    ner = nlp.add_pipe("ner", last=True)
else:
    ner = nlp.get_pipe("ner")

# Add labels to the NER component
for item in train_data:
    for _, _, label in item['entities']:
        ner.add_label(label)

# Prepare training data in the format required by spaCy 3.x
train_examples = []
count=0
for item in train_data:
    doc = nlp.make_doc(item["text"])
    ents = []
    for start, end, label in item['entities']:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is not None:
            ents.append(span)
    
    filtered_ents = filter_spans(ents)
    doc.ents = filtered_ents
    example = Example.from_dict(doc, {"entities": item['entities']})
    train_examples.append(example)



In [4]:
# Initialize the optimizer
optimizer = nlp.begin_training()

# Training loop
n_iter = 300
for itn in range(n_iter):
    random.shuffle(train_examples)
    losses = {}
    # Batch up the examples using spaCy's minibatch
    batches = minibatch(train_examples, size=compounding(4.0, 32.0, 1.001))
    for batch in batches:
        nlp.update(
            batch,  # batch of Example objects
            drop=0.2,  # dropout - make it harder to memorise data
            sgd=optimizer,  # callable to update weights
            losses=losses
        )
    scores = nlp.evaluate(train_examples)
    ents_p = scores["ents_p"]
    ents_r = scores["ents_r"]
    ents_f = scores["ents_f"]

    print(f"Iteration {itn+1}: Losses: {losses['ner']:.3f}, Precision: {ents_p:.3f}, Recall: {ents_r:.3f}, F1-score: {ents_f:.3f}")

# Save the model
nlp.to_disk("ner_model")

Iteration 1: Losses: 26612.209, Precision: 0.997, Recall: 1.000, F1-score: 0.999
Iteration 2: Losses: 5704.469, Precision: 0.934, Recall: 1.000, F1-score: 0.966
Iteration 3: Losses: 3952.015, Precision: 0.823, Recall: 1.000, F1-score: 0.903
Iteration 4: Losses: 3180.710, Precision: 0.900, Recall: 1.000, F1-score: 0.947
Iteration 5: Losses: 2740.532, Precision: 0.851, Recall: 1.000, F1-score: 0.919
Iteration 6: Losses: 2647.418, Precision: 0.884, Recall: 1.000, F1-score: 0.938
Iteration 7: Losses: 2200.027, Precision: 0.885, Recall: 1.000, F1-score: 0.939
Iteration 8: Losses: 2086.462, Precision: 0.881, Recall: 1.000, F1-score: 0.937
Iteration 9: Losses: 1896.823, Precision: 0.893, Recall: 1.000, F1-score: 0.944
Iteration 10: Losses: 1817.931, Precision: 0.931, Recall: 1.000, F1-score: 0.964
Iteration 11: Losses: 1680.189, Precision: 0.936, Recall: 1.000, F1-score: 0.967
Iteration 12: Losses: 1707.834, Precision: 0.907, Recall: 1.000, F1-score: 0.951
Iteration 13: Losses: 1603.189, Prec